In [1]:
import pandas as pd
import duckdb
import psycopg2
import string
import random
from faker import Faker
import pinyin
from datetime import datetime, timedelta
fake = Faker('zh_TW')

# Helper function to generate random timestamps
def random_timestamp(start_date, end_date):
    delta = end_date - start_date
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start_date + timedelta(seconds=random_seconds)

num_items = 3000
num_users = 5000
num_locations = 50
num_categories = 50
num_feedbacks = 500 
num_notifies = 100
num_comments = 1000 
num_messages = 100

### 1. 連線資料庫

In [2]:
with open('db_password.txt', 'r') as file:
    db_password = file.read().strip()

psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)

table_names = ['item', 'lost_item', 'found_item', 'users', 'locations', 'locates', 'stores', 'returns_or_claims', 'category', 'belongs', 'posts', 'members', 'comments', 'message', 'feedback', 'notifies', 'reward'] 
con = duckdb.connect()

for table_name in table_names:
    query_str = "SELECT * FROM " + table_name
    df = pd.read_sql_query(query_str, psql_conn)
    con.register(table_name, df)

psql_conn.close()

/var/folders/y6/fdlf2f113hdd9mwtf7h1m_2r0000gn/T/ipykernel_3509/2643837570.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folders/y6/fdlf2f113hdd9mwtf7h1m_2r0000gn/T/ipykernel_3509/2643837570.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folders/y6/fdlf2f113hdd9mwtf7h1m_2r0000gn/T/ipykernel_3509/2643837570.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folde

In [3]:
'''
# 建立 PostgreSQL 連線
psql_conn = psycopg2.connect(
    dbname='DB_FP',
    user='postgres',
    host='localhost',
    password=db_password
)

# 表名列表
table_names = [
    'item', 'lost_item', 'found_item', 'users', 'locations', 'locates', 
    'stores', 'returns_or_claims', 'category', 'belongs', 'posts', 'members', 
    'comments', 'message', 'feedback', 'notifies', 'reward'
]

# 建立游標
cursor = psql_conn.cursor()

# 使用 DELETE FROM 刪除表中的內容
for table_name in table_names:
    delete_query = f"DELETE FROM {table_name};"
    cursor.execute(delete_query)
    print(f"Deleted data from table: {table_name}")

# 提交更改
psql_conn.commit()

# 關閉游標和連線
cursor.close()
psql_conn.close()
print("All tables have been cleared.")'''

'\n# 建立 PostgreSQL 連線\npsql_conn = psycopg2.connect(\n    dbname=\'DB_FP\',\n    user=\'postgres\',\n    host=\'localhost\',\n    password=db_password\n)\n\n# 表名列表\ntable_names = [\n    \'item\', \'lost_item\', \'found_item\', \'users\', \'locations\', \'locates\', \n    \'stores\', \'returns_or_claims\', \'category\', \'belongs\', \'posts\', \'members\', \n    \'comments\', \'message\', \'feedback\', \'notifies\', \'reward\'\n]\n\n# 建立游標\ncursor = psql_conn.cursor()\n\n# 使用 DELETE FROM 刪除表中的內容\nfor table_name in table_names:\n    delete_query = f"DELETE FROM {table_name};"\n    cursor.execute(delete_query)\n    print(f"Deleted data from table: {table_name}")\n\n# 提交更改\npsql_conn.commit()\n\n# 關閉游標和連線\ncursor.close()\npsql_conn.close()\nprint("All tables have been cleared.")'

In [4]:
# function that sees SQL command AS input and output table from database
def query(query):
    result = con.execute(query).fetchall()
    column_names = [desc[0] for desc in con.description]
    return column_names, result

### 2. Item

In [5]:
adjectives = [
    "粉色的", "藍色的", "綠色的", "黃色的", "紫色的", "紅色的", "白色的", "黑色的", 
    "金色的", "銀色的", "透明的", "可愛的", "簡約的", "豪華的", "復古的", "小巧的",
    "實用的", "新穎的", "時尚的", "高科技的", "耐用的", "輕便的", "便攜的", "環保的",
    "自動的", "迷你的", "手工的", "獨特的", "傳統的", "現代的"
]

nouns = [
    "水壺", "筆記本", "手機殼", "手錶", "包包", "帽子", "雨傘", "耳機",
    "眼鏡", "口罩", "衣服", "鞋子", "手套", "書本", "文件夾", "筆袋",
    "相機", "鑰匙", "卡片", "玩偶", "杯子", "背包", "滑鼠", "鍵盤",
    "行李箱", "手電筒", "蠟燭", "圍巾", "電腦", "充電器", "吊飾", "徽章",
    "手鏈", "項鍊", "戒指", "印章", "筆", "小刀", "日曆", "掛鐘", 
    "音響", "火柴", "剪刀", "水杯", "錢包", "鬧鐘", "運動鞋", "手冊", 
    "收音機", "地圖", "日記本", "粉餅盒", "眼影盤", "行動電源", "遙控器", 
    "腳踏車", "望遠鏡", "毛巾", "電動牙刷", "枕頭", "毛毯", "衣架", "雨靴",
    "學生證", "鏡子", "梳子"
]

item_descriptions = []
while len(item_descriptions) < num_items:
    item_description = f"{random.choice(adjectives)}{random.choice(nouns)}"
    item_descriptions.append(item_description)


items = [
    {
        "ItemID": f"IT{str(i).zfill(8)}",
        "Description": item_descriptions[i - 1],
        "ImageURL": fake.image_url()
    }
    for i in range(1, num_items + 1)
]
items[0:5]

[{'ItemID': 'IT00000001',
  'Description': '粉色的書本',
  'ImageURL': 'https://picsum.photos/819/1'},
 {'ItemID': 'IT00000002',
  'Description': '簡約的水杯',
  'ImageURL': 'https://picsum.photos/420/311'},
 {'ItemID': 'IT00000003',
  'Description': '白色的手冊',
  'ImageURL': 'https://placekitten.com/592/735'},
 {'ItemID': 'IT00000004',
  'Description': '復古的水杯',
  'ImageURL': 'https://placekitten.com/621/131'},
 {'ItemID': 'IT00000005',
  'Description': '高科技的手鏈',
  'ImageURL': 'https://placekitten.com/541/242'}]

In [6]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert data into the 'item' table
for item in items:
    # Create the SQL insert statement
    insert_query = """
    INSERT INTO item (ItemID, Description, ImageURL) 
    VALUES (%s, %s, %s)
    """
    # Execute the insert statement
    cursor.execute(insert_query, (item['ItemID'], item['Description'], item['ImageURL']))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 3. Users

In [7]:
# 姓氏和名字的選項
last_name_chars = [
    "王", "李", "陳", "林", "黃", "張", "吳", "劉", "蔡", "楊", "許", "鄭", "謝", "郭", "洪", "曾", "東方",
    "馮", "方", "朱", "肖", "田", "孔", "董", "崔", "康", "曹", "程", "梁", "羅", "沈", "蘇", "馬", "上官",
    "嚴", "阮", "侯", "邵", "魏", "潘", "盧", "譚", "廖", "范", "杜", "阮", "邱", "高", "葉", "孫", "司馬",
    "江", "毛", "余", "錢", "戴", "柳", "卓", "宗", "龔", "莫", "夏", "冉", "滕", "武", "甄", "歐陽", "皇甫",
    "龍", "郝", "梅", "韓", "秦", "柴", "易", "辛", "紀", "古", "尤", "巴", "陸", "單", "唐", "端木", "諸葛"
]

first_name_chars = [
    "俊", "明", "婌", "美", "佳", "玲", "志", "偉", "建", "宏", "婷", "宇", "軒", "家", "豪", "怡", "鐸",
    "君", "靜", "宜", "傑", "淑", "芬", "冠", "霖", "雅", "詩", "涵", "欣", "柏", "宇", "嘉", "豪", "嵩", 
    "惠", "玲", "輝", "國", "峰", "心", "潔", "慧", "如", "沛", "蓉", "韋", "廷", "瑞", "昌", "艾", "松",
    "莉", "綺", "妍", "宗", "翰", "昱", "廷", "承", "熙", "育", "達", "海", "倫", "雯", "婷", "善", "盈", 
    "思", "新", "玉", "俞", "瑩", "元", "光", "陽", "昀", "哲", "茜", "誠", "曉", "晴", "芮", "可", "卜",
    "欣", "暘", "如", "靈", "蕙", "泓", "葦", "祐", "桐", "榆", "舟", "澄", "修", "恆", "凌", "笙", "壬"
]

# 用戶數據生成
users = []
usernames = set()  # 用於檢查重複名字

while len(users) < num_users:
    # 隨機生成單字姓氏
    last_name = random.choice(last_name_chars)
    
    # 隨機生成名字（兩個字）
    first_name = random.choice(first_name_chars) + random.choice(first_name_chars)
    
    # 全名
    full_name = f"{last_name}{first_name}"
    
    # 檢查是否已存在
    if full_name in usernames:
        continue  # 如果名字重複，跳過此次生成
    
    # 添加到集合中以避免重複
    usernames.add(full_name)
    
    # 姓名轉拼音
    last_name_pinyin = pinyin.get(last_name, format="strip", delimiter="")
    first_name_pinyin = pinyin.get(first_name, format="strip", delimiter="")
    
    # 隨機生成唯一 Email
    email = f"{last_name_pinyin}{first_name_pinyin}{random.randint(1, 99)}@gmail.com"
    
    # 組成用戶數據
    users.append({
        "UserID": f"US{str(len(users) + 1).zfill(8)}",
        "UserName": full_name,
        "Email": email,
        "PhoneNumber": f"09{random.randint(10000000, 99999999)}"
    })

users[:5]


[{'UserID': 'US00000001',
  'UserName': '滕怡芬',
  'Email': 'tengyifen87@gmail.com',
  'PhoneNumber': '0994569817'},
 {'UserID': 'US00000002',
  'UserName': '廖欣芮',
  'Email': 'liaoxinrui79@gmail.com',
  'PhoneNumber': '0973768715'},
 {'UserID': 'US00000003',
  'UserName': '毛雯美',
  'Email': 'maowenmei99@gmail.com',
  'PhoneNumber': '0986493421'},
 {'UserID': 'US00000004',
  'UserName': '林嘉育',
  'Email': 'linjiayu7@gmail.com',
  'PhoneNumber': '0991371851'},
 {'UserID': 'US00000005',
  'UserName': '巴俞海',
  'Email': 'bayuhai21@gmail.com',
  'PhoneNumber': '0917666846'}]

In [8]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert data into the 'users' table
for user in users:
    # Create the SQL insert statement
    insert_query = """
    INSERT INTO users (UserID, UserName, Email, PhoneNumber) 
    VALUES (%s, %s, %s, %s)
    """
    # Execute the insert statement
    cursor.execute(insert_query, (user['UserID'], user['UserName'], user['Email'], user['PhoneNumber']))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 4. Locations

In [9]:
campus_locations = [
    "台大正門", "椰林大道", "台大校史館", "總圖書館", "電機一館", "電機二館", "管理學院一館", "管理學院二館", 
    "教研館", "計算機及資訊網路中心", "思亮館", "新月台", "理學院", "普通教學館", "綜合體育館", "小福樓", "台大農場", 
    "學生活動中心", "化學系館", "舟山路前段", "舟山路後段", "學新館", "第二學生活動中心", "鹿鳴堂", "游泳池", 
    "博雅教學館", "國青宿舍", "醉月湖", "德田館", "女九舍", "駐警隊"
]

nearby_locations = [
    "公館夜市", "溫州街", "羅斯福路四段", "台電大樓捷運站", "科技大樓捷運站", "和平東路", "基隆路", 
    "公館捷運站一號出口", "公館捷運站二號出口", "公館捷運站三號出口", "公館捷運站四號出口", "新生南路", 
    "師大夜市", "公館水岸公園", "水源市場", "公館捷運站"
]

# 生成地點數據
locations = []

# 先添加所有校內地點
for i, location in enumerate(campus_locations, start=1):
    locations.append({
        "LocationID": f"LO{str(i).zfill(8)}",
        "LocationDescription": location,
        "IsInCampus": True
    })

# 再添加所有校外地點
for i, location in enumerate(nearby_locations, start=len(campus_locations) + 1):
    locations.append({
        "LocationID": f"LO{str(i).zfill(8)}",
        "LocationDescription": location,
        "IsInCampus": False
    })

# 輸出所有的地點示例
locations[0:5]  # 可調整顯示的數量


[{'LocationID': 'LO00000001',
  'LocationDescription': '台大正門',
  'IsInCampus': True},
 {'LocationID': 'LO00000002',
  'LocationDescription': '椰林大道',
  'IsInCampus': True},
 {'LocationID': 'LO00000003',
  'LocationDescription': '台大校史館',
  'IsInCampus': True},
 {'LocationID': 'LO00000004',
  'LocationDescription': '總圖書館',
  'IsInCampus': True},
 {'LocationID': 'LO00000005',
  'LocationDescription': '電機一館',
  'IsInCampus': True}]

In [10]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert data into the 'locations' table
for location in locations:
    # Create the SQL insert statement
    insert_query = """
    INSERT INTO locations (LocationID, LocationDescription, IsInCampus) 
    VALUES (%s, %s, %s)
    """
    # Execute the insert statement
    cursor.execute(insert_query, (location['LocationID'], location['LocationDescription'], location['IsInCampus']))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 5. Category

In [11]:
# 預定義類別名稱
category_names = [
    "文具", "衣物", "配件", "電子產品", "交通工具", "居家用品", "個人護理", "運動用品", "收藏品", "其他"
]

num_categories = len(category_names)  # 根據預設類別數生成
categories = [
    {
        "CategoryID": f"CA{str(i).zfill(8)}",
        "CategoryName": category_names[i - 1]
    }
    for i in range(1, num_categories + 1)
]

categories[0:5]

[{'CategoryID': 'CA00000001', 'CategoryName': '文具'},
 {'CategoryID': 'CA00000002', 'CategoryName': '衣物'},
 {'CategoryID': 'CA00000003', 'CategoryName': '配件'},
 {'CategoryID': 'CA00000004', 'CategoryName': '電子產品'},
 {'CategoryID': 'CA00000005', 'CategoryName': '交通工具'}]

In [12]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert data into the 'category' table
for category in categories:
    # Create the SQL insert statement
    insert_query = """
    INSERT INTO category (CategoryID, CategoryName) 
    VALUES (%s, %s)
    """
    # Execute the insert statement
    cursor.execute(insert_query, (category['CategoryID'], category['CategoryName']))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 6 & 7. lost_items found_items

In [13]:
num_items = len(items)

# 將 items 的 ID 隨機打亂
shuffled_items = random.sample(items, num_items)

# 隨機分配 LostItems 和 FoundItems
lost_items = []
found_items = []

# 隨機分割比例，50% 的機率成為 LostItem 或 FoundItem
for item in shuffled_items:
    if random.random() < 0.5:  # 50% 的機率作為 LostItem
        lost_items.append(
            {
                "ItemID": item["ItemID"],
                "LostTime": random_timestamp(datetime(2023, 1, 1), datetime(2024, 12, 1)),
            }
        )
    else:  # 否則作為 FoundItem
        found_items.append(
            {
                "ItemID": item["ItemID"],
                "FoundTime": random_timestamp(datetime(2023, 1, 1), datetime(2024, 12, 1)),
            }
        )
lost_items[0:5], found_items[0:5]


([{'ItemID': 'IT00002373',
   'LostTime': datetime.datetime(2023, 5, 8, 22, 3, 29)},
  {'ItemID': 'IT00000545',
   'LostTime': datetime.datetime(2023, 11, 17, 15, 54, 57)},
  {'ItemID': 'IT00000351',
   'LostTime': datetime.datetime(2023, 12, 15, 6, 24, 8)},
  {'ItemID': 'IT00001007',
   'LostTime': datetime.datetime(2024, 10, 19, 21, 53, 30)},
  {'ItemID': 'IT00001168',
   'LostTime': datetime.datetime(2023, 6, 26, 7, 37, 56)}],
 [{'ItemID': 'IT00000669',
   'FoundTime': datetime.datetime(2023, 11, 20, 15, 45, 12)},
  {'ItemID': 'IT00001905',
   'FoundTime': datetime.datetime(2024, 10, 31, 22, 25, 8)},
  {'ItemID': 'IT00001381',
   'FoundTime': datetime.datetime(2024, 8, 25, 22, 23, 51)},
  {'ItemID': 'IT00001869',
   'FoundTime': datetime.datetime(2023, 2, 23, 2, 47, 16)},
  {'ItemID': 'IT00001480',
   'FoundTime': datetime.datetime(2024, 4, 11, 16, 22, 50)}])

In [14]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert Lost Items
for lost_item in lost_items:
    # Create the SQL insert statement for LostItem
    insert_query = """
    INSERT INTO lost_item (ItemID, LostTime) 
    VALUES (%s, %s)
    """
    # Execute the insert statement for LostItem
    cursor.execute(insert_query, (lost_item["ItemID"], lost_item["LostTime"]))

# Insert Found Items
for found_item in found_items:
    # Create the SQL insert statement for FoundItem
    insert_query = """
    INSERT INTO found_item (ItemID, FoundTime) 
    VALUES (%s, %s)
    """
    # Execute the insert statement for FoundItem
    cursor.execute(insert_query, (found_item["ItemID"], found_item["FoundTime"]))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 8. Belongs

In [15]:
# 類別與關鍵詞的對應關係
category_keywords = {
    "文具": ["筆記本", "文件夾", "筆袋", "印章", "日曆", "書本"],
    "衣物": ["衣服", "鞋子", "圍巾", "手套", "帽子", "毛毯"],
    "配件": ["手錶", "耳機", "眼鏡", "項鍊", "戒指", "手鏈"],
    "電子產品": ["手機殼", "相機", "電腦", "鍵盤", "滑鼠", "充電器", "行動電源"],
    "交通工具": ["腳踏車"],
    "居家用品": ["枕頭", "衣架", "掛鐘", "日記本", "毛巾"],
    "個人護理": ["梳子", "口罩", "粉餅盒", "眼影盤", "電動牙刷"],
    "運動用品": ["運動鞋", "背包", "雨傘"],
    "收藏品": ["吊飾", "徽章", "玩偶"],
    "其他": []
}

# 將類別名稱轉換為對應的 CategoryID
category_name_to_id = {category["CategoryName"]: category["CategoryID"] for category in categories}

# 根據物品描述決定所屬類別
belongs = []
for item in items:
    item_description = item["Description"]
    matched_category = "其他"  # 預設為 "其他"
    for category_name, keywords in category_keywords.items():
        if any(keyword in item_description for keyword in keywords):
            matched_category = category_name
            break
    belongs.append({
        "ItemID": item["ItemID"],
        "CategoryID": category_name_to_id[matched_category]
    })

belongs[0:5]


[{'ItemID': 'IT00000001', 'CategoryID': 'CA00000001'},
 {'ItemID': 'IT00000002', 'CategoryID': 'CA00000010'},
 {'ItemID': 'IT00000003', 'CategoryID': 'CA00000010'},
 {'ItemID': 'IT00000004', 'CategoryID': 'CA00000010'},
 {'ItemID': 'IT00000005', 'CategoryID': 'CA00000003'}]

In [16]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert into 'belongs' table
for belong in belongs:
    # Create the SQL insert statement for the 'belongs' table
    insert_query = """
    INSERT INTO belongs (ItemID, CategoryID) 
    VALUES (%s, %s)
    """
    # Execute the insert statement
    cursor.execute(insert_query, (belong["ItemID"], belong["CategoryID"]))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 9. Posts

In [17]:
# 生成 POST 數據
posts = []

# 從所有物品中隨機選擇一個物品，並為每個物品生成一個唯一的貼文
used_items = set()  # 用來記錄已經使用過的 ItemID，保證每個 ItemID 唯一
for item in items:
    # 隨機選擇一個 UserID 發佈貼文
    user = random.choice(users)
    
    # 確保 ItemID 唯一
    while item["ItemID"] in used_items:
        item = random.choice(items)  # 隨機選擇新的物品，直到選到未使用過的
    
    # 記錄該 ItemID 已經被使用
    used_items.add(item["ItemID"])

    # 隨機生成發佈時間
    post_time = random_timestamp(datetime(2023, 1, 1), datetime(2024, 12, 1))
    
    # 添加貼文數據
    posts.append(
        {
            "UserID": user["UserID"],
            "ItemID": item["ItemID"],
            "PostTime": post_time,
        }
    )

# 顯示前五筆數據
posts[0:5]

[{'UserID': 'US00004387',
  'ItemID': 'IT00000001',
  'PostTime': datetime.datetime(2023, 6, 25, 9, 24, 39)},
 {'UserID': 'US00001596',
  'ItemID': 'IT00000002',
  'PostTime': datetime.datetime(2024, 3, 11, 20, 6, 58)},
 {'UserID': 'US00004511',
  'ItemID': 'IT00000003',
  'PostTime': datetime.datetime(2023, 5, 16, 5, 43, 35)},
 {'UserID': 'US00004997',
  'ItemID': 'IT00000004',
  'PostTime': datetime.datetime(2023, 3, 13, 16, 15, 45)},
 {'UserID': 'US00000761',
  'ItemID': 'IT00000005',
  'PostTime': datetime.datetime(2024, 8, 29, 10, 58, 17)}]

In [18]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert into 'posts' table
for post in posts:
    # Create the SQL insert statement for the 'posts' table
    insert_query = """
    INSERT INTO posts (UserID, ItemID, PostTime) 
    VALUES (%s, %s, %s)
    """
    # Execute the insert statement
    cursor.execute(insert_query, (post["UserID"], post["ItemID"], post["PostTime"]))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 10 & 11. Stores & Locates

In [19]:
stores = []
locates = []
for item in items:
    location = random.choice(locations)  # 隨機選擇一個地點
    start_time = random_timestamp(datetime(2023, 1, 1), post_time)
    if random.random() < 0.8:  # 假設有80%的機會設置 finish_time
        finish_time = random_timestamp(start_time, start_time + timedelta(days=10))
    else:
        finish_time = None 

    # 強制設定存放位置，校內放駐警隊，校外放捷運站
    if location["IsInCampus"]:
        # 如果是在校內，強制存放在「駐警隊」
        location_store = "駐警隊"
    else:
        # 如果是校外，強制存放在「公館捷運站」
        location_store = "公館捷運站"

    location_store_id = next(loc["LocationID"] for loc in locations if loc["LocationDescription"] == location_store)
     
    stores.append(
        {
            "ItemID": item["ItemID"],
            "LocationID": location_store_id,
            "StartTime": start_time,
            "FinishTime": finish_time,
        }
    )
    locates.append(
        {
            "ItemID": item["ItemID"],
            "LocationID": location["LocationID"],
        }
    )

# 輸出部分示例
stores[0:5]


[{'ItemID': 'IT00000001',
  'LocationID': 'LO00000031',
  'StartTime': datetime.datetime(2023, 1, 22, 2, 6, 50),
  'FinishTime': datetime.datetime(2023, 1, 25, 9, 51)},
 {'ItemID': 'IT00000002',
  'LocationID': 'LO00000031',
  'StartTime': datetime.datetime(2023, 1, 10, 2, 42, 8),
  'FinishTime': datetime.datetime(2023, 1, 18, 7, 42, 38)},
 {'ItemID': 'IT00000003',
  'LocationID': 'LO00000031',
  'StartTime': datetime.datetime(2023, 7, 4, 22, 42, 26),
  'FinishTime': datetime.datetime(2023, 7, 12, 23, 22, 9)},
 {'ItemID': 'IT00000004',
  'LocationID': 'LO00000031',
  'StartTime': datetime.datetime(2023, 1, 31, 13, 3, 27),
  'FinishTime': datetime.datetime(2023, 2, 6, 3, 13, 46)},
 {'ItemID': 'IT00000005',
  'LocationID': 'LO00000047',
  'StartTime': datetime.datetime(2023, 5, 16, 14, 6, 13),
  'FinishTime': None}]

In [20]:
locates[0:5]

[{'ItemID': 'IT00000001', 'LocationID': 'LO00000019'},
 {'ItemID': 'IT00000002', 'LocationID': 'LO00000006'},
 {'ItemID': 'IT00000003', 'LocationID': 'LO00000017'},
 {'ItemID': 'IT00000004', 'LocationID': 'LO00000026'},
 {'ItemID': 'IT00000005', 'LocationID': 'LO00000039'}]

In [21]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert into 'stores' table
for store in stores:
    insert_query = """
    INSERT INTO stores (ItemID, LocationID, StartTime, FinishTime) 
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(insert_query, (store["ItemID"], store["LocationID"], store["StartTime"], store["FinishTime"]))

# Insert into 'locates' table
for locate in locates:
    insert_query = """
    INSERT INTO locates (ItemID, LocationID) 
    VALUES (%s, %s)
    """
    cursor.execute(insert_query, (locate["ItemID"], locate["LocationID"]))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 12. Returns_or_Claims

In [22]:
# 記錄每個物品的狀態變化
item_status = {}

# 儲存物品的發佈時間
post_times = {item["ItemID"]: random.choice(posts)["PostTime"] for item in items}

returns_or_claims = []

while len(returns_or_claims) < 10000:
    item = random.choice(items)
    user = random.choice(users)
    post_time = post_times[item["ItemID"]]  # 獲取該物品的 PostTime
    
    # 隨機生成一個 RCTime，必須在 PostTime 之後
    rctime = random_timestamp(post_time, datetime(2024, 12, 1))
    
    # 如果這個物品還沒有開始處理過狀態，初始化為進行中
    if item["ItemID"] not in item_status:
        item_status[item["ItemID"]] = ["P"]  # 初始化為進行中
    
    # 如果物品狀態已經是完成 (S)，跳過
    if item_status[item["ItemID"]][-1] == "S":
        continue  # 如果物品已經完成，不再添加新的狀態
    
    # 根據最後一次狀態決定下一個狀態
    last_status = item_status[item["ItemID"]][-1]
    
    # 如果狀態是進行中，隨機決定下一步
    if last_status == "P":
        # 約50%完成，30%取消，20%失敗
        if random.random() < 0.5:  # 完成
            status = "S"
        elif random.random() < 0.3:  # 取消
            status = "C"
        else:  # 失敗
            status = "F"
    # 如果狀態是完成或取消/失敗，則不會再改為進行中，並且完成會是最後狀態
    else:
        status = last_status  # 取消或失敗，保持不變
    
    # 添加狀態到物品的狀態序列中
    item_status[item["ItemID"]].append(status)

    returns_or_claims.append(
        {
            "ItemID": item["ItemID"],
            "UserID": user["UserID"],
            "RCTime": rctime,
            "Status": status,
        }
    )

# 按照 RCTime 排序
returns_or_claims.sort(key=lambda x: x["RCTime"])

# 顯示前五筆數據
returns_or_claims[0:5]


[{'ItemID': 'IT00001421',
  'UserID': 'US00004416',
  'RCTime': datetime.datetime(2023, 1, 16, 16, 2, 24),
  'Status': 'F'},
 {'ItemID': 'IT00002413',
  'UserID': 'US00004617',
  'RCTime': datetime.datetime(2023, 1, 16, 22, 45, 23),
  'Status': 'C'},
 {'ItemID': 'IT00000429',
  'UserID': 'US00000945',
  'RCTime': datetime.datetime(2023, 1, 21, 7, 35, 22),
  'Status': 'F'},
 {'ItemID': 'IT00002144',
  'UserID': 'US00001303',
  'RCTime': datetime.datetime(2023, 1, 22, 15, 28, 55),
  'Status': 'C'},
 {'ItemID': 'IT00002948',
  'UserID': 'US00002494',
  'RCTime': datetime.datetime(2023, 1, 24, 23, 9, 39),
  'Status': 'F'}]

In [23]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert into 'returns_or_claims' table
for record in returns_or_claims:
    insert_query = """
    INSERT INTO returns_or_claims (ItemID, UserID, RCTime, Status) 
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(insert_query, (record["ItemID"], record["UserID"], record["RCTime"], record["Status"]))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 13. Notifies

In [24]:
# 記錄每個物品和用戶的通知數據
notifies = []


# 生成通知數據
for _ in range(num_notifies):
    item = random.choice(items)  # 隨機選擇物品
    user = random.choice(users)  # 隨機選擇用戶
    notify_time = random_timestamp(post_time, datetime(2024, 12, 1)) # 隨機生成通知時間

    # 添加通知數據
    notifies.append(
        {
            "ItemID": item["ItemID"],
            "UserID": user["UserID"],
            "NotifyTime": notify_time,
        }
    )

# 按照 NotifyTime 排序
notifies.sort(key=lambda x: x["NotifyTime"])

# 顯示前五筆通知數據
notifies[0:5]

[{'ItemID': 'IT00002690',
  'UserID': 'US00001557',
  'NotifyTime': datetime.datetime(2024, 11, 4, 10, 6, 33)},
 {'ItemID': 'IT00001593',
  'UserID': 'US00000227',
  'NotifyTime': datetime.datetime(2024, 11, 4, 14, 49, 57)},
 {'ItemID': 'IT00001609',
  'UserID': 'US00002775',
  'NotifyTime': datetime.datetime(2024, 11, 4, 21, 53, 18)},
 {'ItemID': 'IT00001260',
  'UserID': 'US00002287',
  'NotifyTime': datetime.datetime(2024, 11, 5, 2, 30, 5)},
 {'ItemID': 'IT00000593',
  'UserID': 'US00002164',
  'NotifyTime': datetime.datetime(2024, 11, 5, 7, 23, 8)}]

In [25]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert into 'notifies' table
for record in notifies:
    insert_query = """
    INSERT INTO notifies (ItemID, UserID, NotifyTime) 
    VALUES (%s, %s, %s)
    """
    cursor.execute(insert_query, (record["ItemID"], record["UserID"], record["NotifyTime"]))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 14. Rewards

In [26]:
# 定義可能的懸賞名稱
reward_names = [
    "星巴克", "100元現金", "手搖杯", "電子禮品卡", "超商禮券", "電影票"
]

# 隨機選擇 100 個物品
reward_items = random.sample(items, 100)

# 生成獎勳數據
rewards = []

for item in reward_items:
    reward_name = random.choice(reward_names)  # 隨機選擇懸賞名稱
    amount = random.randint(1, 3)  # 隨機選擇懸賞數量，1-3杯/張
    rewards.append(
        {
            "ItemID": item["ItemID"],
            "RewardName": reward_name,
            "Amount": amount,
        }
    )

# 顯示前五筆獎勳數據
rewards[0:5]


[{'ItemID': 'IT00002328', 'RewardName': '星巴克', 'Amount': 2},
 {'ItemID': 'IT00000882', 'RewardName': '手搖杯', 'Amount': 1},
 {'ItemID': 'IT00002214', 'RewardName': '電影票', 'Amount': 1},
 {'ItemID': 'IT00001011', 'RewardName': '星巴克', 'Amount': 3},
 {'ItemID': 'IT00000322', 'RewardName': '超商禮券', 'Amount': 3}]

In [27]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert into 'rewards' table
for record in rewards:
    insert_query = """
    INSERT INTO reward (ItemID, RewardName, Amount) 
    VALUES (%s, %s, %s)
    """
    cursor.execute(insert_query, (record["ItemID"], record["RewardName"], record["Amount"]))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 15. Members

In [28]:
# 假設 users 表中已經有足夠的用戶數據
# 從 users 表中選擇 1000 個不同的 UserID
sample_users = random.sample(users, 1000)

# 生成 member 的數據
members = []
for user in sample_users:
    # 隨機生成帳號名稱
    account_name = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))  # 隨機 8 個字母數字組合
    # 隨機生成密碼
    password = ''.join(random.choices(string.ascii_letters + string.digits, k=12))  # 隨機 12 個字母數字組合
    
    members.append(
        {
            "MemberID": user["UserID"],
            "AccountName": account_name,
            "Password": password,
        }
    )

# 顯示前五筆生成的會員數據
members[0:5]


[{'MemberID': 'US00004569',
  'AccountName': 'zuql2nwe',
  'Password': 'm19ZoF12mhBt'},
 {'MemberID': 'US00003495',
  'AccountName': '7cnx0lu4',
  'Password': 'SGSMlWP0ROzU'},
 {'MemberID': 'US00004346',
  'AccountName': 'j45emcg3',
  'Password': 'NLymAGmQo7Xv'},
 {'MemberID': 'US00003923',
  'AccountName': 'njbjaut5',
  'Password': 'YQT5pjlUZnPb'},
 {'MemberID': 'US00000906',
  'AccountName': 'rv3iftvu',
  'Password': 'SpDACpQCT23z'}]

In [29]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert into 'members' table
for record in members:
    insert_query = """
    INSERT INTO members (MemberID, AccountName, Password) 
    VALUES (%s, %s, %s)
    """
    cursor.execute(insert_query, (record["MemberID"], record["AccountName"], record["Password"]))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 16. Comments

In [30]:
comments_templates = [
    "謝謝你撿到！真的太感謝了！",
    "感謝好心人找到了我的東西！",
    "真的很感謝，這個物品對我很重要！",
    "謝謝你，我終於找回來了！",
    "太感謝了！我終於找到了，謝謝你！",
    "我記得放在椅子上，請問可以再提供一些細節嗎？",
    "請問這是我的東西嗎？我記得掉在桌子上。",
    "我的丟失了，請問這是我的嗎？",
    "我弄丟了它，請問可以提供更詳細的描述嗎？",
    "我丟了它請問在哪裡找回來？",
    "這是我弄丟的，請問是在哪個地點找到的呢？",
    "謝謝你撿到我的物品！我會去指定地點領取。",
    "謝謝你的幫忙，已經去領回物品了，感恩！",
    "感謝找到我的筆記本，已經取回了！",
    "謝謝你撿到我的耳機，我已經領回來了。",
    "非常感謝，你讓我找回了我的東西，我會盡快來領取！"
]


# 隨機生成留言數據
comments = []

for _ in range(num_comments):
    item = random.choice(items)  # 隨機選擇物品
    member = random.choice(members)  # 隨機選擇用戶
    cm_time = random_timestamp(post_time, datetime(2024, 12, 1))
    cm_content = random.choice(comments_templates)  # 隨機選擇留言內容
    
    comments.append({
        "MemberID": member["MemberID"],
        "ItemID": item["ItemID"],
        "CmTime": cm_time,
        "CmContent": cm_content
    })

# 輸出部分示例
comments[0:5]  # 顯示前5條留言

[{'MemberID': 'US00004191',
  'ItemID': 'IT00000416',
  'CmTime': datetime.datetime(2024, 11, 20, 9, 13, 19),
  'CmContent': '感謝找到我的筆記本，已經取回了！'},
 {'MemberID': 'US00003445',
  'ItemID': 'IT00001879',
  'CmTime': datetime.datetime(2024, 11, 13, 4, 6, 49),
  'CmContent': '感謝找到我的筆記本，已經取回了！'},
 {'MemberID': 'US00002286',
  'ItemID': 'IT00002448',
  'CmTime': datetime.datetime(2024, 11, 27, 3, 31, 36),
  'CmContent': '這是我弄丟的，請問是在哪個地點找到的呢？'},
 {'MemberID': 'US00004035',
  'ItemID': 'IT00000771',
  'CmTime': datetime.datetime(2024, 11, 11, 6, 57, 2),
  'CmContent': '我的丟失了，請問這是我的嗎？'},
 {'MemberID': 'US00002093',
  'ItemID': 'IT00000369',
  'CmTime': datetime.datetime(2024, 11, 13, 0, 59, 36),
  'CmContent': '謝謝你，我終於找回來了！'}]

In [31]:
# Connect to PostgreSQL database
psql_conn = psycopg2.connect("dbname = 'DB_FP' user = 'postgres' host = 'localhost' password = " + db_password)
cursor = psql_conn.cursor()

# Insert into 'comments' table
for record in comments:
    insert_query = """
    INSERT INTO comments (MemberID, ItemID, CmTime, CmContent) 
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(insert_query, (record["MemberID"], record["ItemID"], record["CmTime"], record["CmContent"]))

# Commit the transaction
psql_conn.commit()

# Close the cursor and connection
cursor.close()
psql_conn.close()

### 17. Messages

In [32]:
# 隨機生成私訊數據
messages = []

for _ in range(num_messages):
    sender = random.choice(members)  # 隨機選擇發送者
    receiver = random.choice(members)  # 隨機選擇接收者
    mg_time =  random_timestamp(post_time, datetime(2024, 12, 1))
    mg_content = random.choice(comments_templates)  # 隨機選擇私訊內容
    
    messages.append({
        "SenderID": sender["MemberID"],
        "ReceiverID": receiver["MemberID"],
        "MgTime": mg_time,
        "MgContent": mg_content
    })

# 輸出部分示例
messages[0:5]  # 顯示前5條私訊


[{'SenderID': 'US00003440',
  'ReceiverID': 'US00003413',
  'MgTime': datetime.datetime(2024, 11, 23, 15, 52, 3),
  'MgContent': '真的很感謝，這個物品對我很重要！'},
 {'SenderID': 'US00002293',
  'ReceiverID': 'US00003159',
  'MgTime': datetime.datetime(2024, 11, 24, 10, 17, 17),
  'MgContent': '我弄丟了它，請問可以提供更詳細的描述嗎？'},
 {'SenderID': 'US00003390',
  'ReceiverID': 'US00002580',
  'MgTime': datetime.datetime(2024, 11, 23, 9, 52, 26),
  'MgContent': '請問這是我的東西嗎？我記得掉在桌子上。'},
 {'SenderID': 'US00001879',
  'ReceiverID': 'US00004720',
  'MgTime': datetime.datetime(2024, 11, 30, 19, 0, 40),
  'MgContent': '我弄丟了它，請問可以提供更詳細的描述嗎？'},
 {'SenderID': 'US00001540',
  'ReceiverID': 'US00001415',
  'MgTime': datetime.datetime(2024, 11, 24, 3, 11, 8),
  'MgContent': '謝謝你撿到！真的太感謝了！'}]

In [33]:
psql_conn = psycopg2.connect("dbname='DB_FP' user='postgres' host='localhost' password=" + db_password)
cursor = psql_conn.cursor()

# 插入數據到 'message' 表
for record in messages:
    insert_query = """
    INSERT INTO message (SenderID, ReceiverID, MgTime, MgContent) 
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(insert_query, (record["SenderID"], record["ReceiverID"], record["MgTime"], record["MgContent"]))

# 提交交易
psql_conn.commit()

# 關閉游標和連接
cursor.close()
psql_conn.close()

### 18. Feedbacks

In [34]:
# 設計回饋的範例內容
feedback_templates = [
    "非常感謝這個系統，讓我找回了丟失的物品！希望大家能夠更積極參與，讓失物招領系統更加完善！",
    "這個系統太棒了！我丟失的耳機找回來了，非常感謝！",
    "希望能夠增加一個自動通知功能，當有相關物品被找到時，系統能及時通知我。",
    "希望能增加地點篩選功能，讓我能夠更精確地查找丟失物品的地點，這樣會更方便。",
    "界面設計得很好，操作簡單明瞭，期待未來能增添更多功能，比如物品自動識別，這樣會更加高效。",
    "希望能在物品描述中添加照片的上傳功能，這樣能幫助我更快地找到丟失的物品。",
    "最近在提交失物信息時，系統有時會顯示錯誤，無法成功提交，請協助處理一下。",
    "有時候更新物品狀態的反應速度較慢，希望能加快系統反應時間。",
    "希望能夠增加一個查詢過去的失物記錄的功能，這樣如果物品再次丟失，我可以查看是否有相似物品被找到過。",
    "希望能增加語音輸入功能，這樣可以更便捷地描述丟失物品，尤其在手忙腳亂的時候很有幫助。",
    "真的非常感謝這個平台，我找回了丟失的背包，這對我來說很重要。已經推薦給我的朋友們了！",
    "這個系統真的是太方便了，謝謝你們幫助我找回了失物。我一定會繼續使用並推薦給更多人！",
]


# 隨機生成回饋數據
feedbacks = []

for _ in range(num_feedbacks):
    user = random.choice(users)  # 隨機選擇用戶ID
    fb_time = random_timestamp(datetime(2023, 1, 1), datetime(2024, 12, 1)) 
    fb_content = random.choice(feedback_templates)  # 隨機選擇回饋內容
    
    feedbacks.append({
        "UserID": user["UserID"],
        "FBTime": fb_time,
        "FBContent": fb_content
    })

# 輸出部分示例
feedbacks[0:5]  # 顯示前5條回饋


[{'UserID': 'US00003527',
  'FBTime': datetime.datetime(2023, 12, 13, 1, 42, 18),
  'FBContent': '希望能增加語音輸入功能，這樣可以更便捷地描述丟失物品，尤其在手忙腳亂的時候很有幫助。'},
 {'UserID': 'US00000370',
  'FBTime': datetime.datetime(2023, 8, 18, 13, 52, 30),
  'FBContent': '非常感謝這個系統，讓我找回了丟失的物品！希望大家能夠更積極參與，讓失物招領系統更加完善！'},
 {'UserID': 'US00001646',
  'FBTime': datetime.datetime(2023, 4, 10, 19, 39, 34),
  'FBContent': '希望能增加語音輸入功能，這樣可以更便捷地描述丟失物品，尤其在手忙腳亂的時候很有幫助。'},
 {'UserID': 'US00000140',
  'FBTime': datetime.datetime(2023, 8, 19, 6, 13, 55),
  'FBContent': '希望能夠增加一個查詢過去的失物記錄的功能，這樣如果物品再次丟失，我可以查看是否有相似物品被找到過。'},
 {'UserID': 'US00003907',
  'FBTime': datetime.datetime(2024, 3, 6, 19, 8, 6),
  'FBContent': '希望能在物品描述中添加照片的上傳功能，這樣能幫助我更快地找到丟失的物品。'}]

In [35]:
psql_conn = psycopg2.connect("dbname='DB_FP' user='postgres' host='localhost' password=" + db_password)
cursor = psql_conn.cursor()

# 插入數據到 'feedback' 表
for record in feedbacks:
    insert_query = """
    INSERT INTO feedback (UserID, FBTime, FBContent) 
    VALUES (%s, %s, %s)
    """
    cursor.execute(insert_query, (record["UserID"], record["FBTime"], record["FBContent"]))

# 提交交易
psql_conn.commit()

# 關閉游標和連接
cursor.close()
psql_conn.close()